In [1]:
import numpy as np
from matplotlib import pyplot as plt
from scipy import interpolate
from CAMB import camb
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [2]:
wmap = np.loadtxt('wmap_tt_spectrum_9yr_v5.txt') #Extracting the data
multipoleIndex = wmap[:,0] #ydata
PP = wmap[:,1] #xdata
errPP = wmap[:,2] #Error on x data
errInst = wmap[:,3]
errCos = wmap[:,4]
pars = [multipoleIndex,PP,errPP,errInst,errCos]

In [3]:
def get_spectrum(pars,lmax=2000, fixed_tau=None):
    #print('pars are ',pars)
    if fixed_tau is None:
        H0=pars[0]
        ombh2=pars[1]
        omch2=pars[2]
        tau=pars[3]
        As=pars[4]
        ns=pars[5]
    else: 
        H0=pars[0]
        ombh2=pars[1]
        omch2=pars[2]
        tau=fixed_tau
        As=pars[3]
        ns=pars[4]
    pars=camb.CAMBparams()
    pars.set_cosmology(H0=H0,ombh2=ombh2,omch2=omch2,mnu=0.06,omk=0,tau=tau)
    pars.InitPower.set_params(As=As,ns=ns,r=0)
    pars.set_for_lmax(lmax,lens_potential_accuracy=0)
    results=camb.get_results(pars)
    powers=results.get_cmb_power_spectra(pars,CMB_unit='muK')
    cmb=powers['total']
    tt=cmb[:,0]
    return tt

Markov-chain Monte Carlo where we fit our 6 parameters.
Note that the optical depth tau cannot be negative; avoid sampling a negative tau.

In [4]:
#Draw samples of correlated data from a covariance matrix
#Taken from Prof. Sievers
def take_step_cov(covmat):
    mychol = np.linalg.cholesky(covmat)
    return np.dot(mychol,np.random.randn(covmat.shape[0]))

#Running an MCMC
def MCMC(x,y,p,errP,nstep,pcov):
        step = 0 #A counter for the step number
        chains = np.zeros([nstep,len(p)]) #The total number of chains to run: given
        chi_sq_vec = np.zeros(nstep) #A vector of the chi^2s
        chi_sq = np.sum((x-y(p)[2:len(x)+2])**2/errP**2) #Computing the chi^2
        scale = 0.5 #Similar to the damping factor in question 3
        for i in range(nstep): #For every "guess" we take
            p_new = p + take_step_cov(pcov)*scale #Slightly vary the parameter that we are at
            if p_new[3]>0: #Ignoring the negative params (tau, in this case) because they are unphysical
                m_new = y(p_new)[2:len(x)+2] #Get the y values for this new parameter
                new_chi_sq = np.sum((x-m_new)**2/errP**2) #Get the new chi^2 with this new parameter
                cond = new_chi_sq - chi_sq #Compute the difference between the old chi^2 and the new one
                prob = np.exp(-0.5*cond) #The gaussian prediction of what the parameter is supposed to be
                accept = np.random.rand(1)<prob #Deciding if the mcmc will go to that point through probability
                if accept: #If the mcmc goes to the point
                    #print('The step has been accepted')
                    p = p_new #Get the new parameter
                    m = m_new #And y value
                    chi_sq = new_chi_sq #And chi^2
                    step+=1 #Increase the step by one 
                    print('We are at step', step, 'and the chi^2 is', chi_sq)
                    #print('The new chi^2 is', chi_sq)
                else:
                    #print('The step has been rejected; choose another step size/direction')
            chains[i,:] = p #Collect all of the parameters that we generate in the chain
            chi_sq_vec[i] = chi_sq #Collect the new chi^2 values too
        return chains,chi_sq_vec,p

In [5]:
#Getting the covariance matrix from question 3, with the unspecified tau
pcov = [[ 3.15489588e+00,  5.33722850e-04, -5.12914585e-03,  2.29292184e-02,
   6.22487886e-11,  1.48144365e-02], [ 5.33722850e-04,  2.81921932e-07, -9.72381229e-08, -1.06025541e-05,
  -3.38511692e-14,  5.85014193e-06], [-5.12914585e-03, -9.72381229e-08,  1.45665908e-05, -9.72396659e-05,
  -2.61420053e-13, -1.19449106e-05], [ 2.29292184e-02, -1.06025541e-05, -9.72396659e-05,  1.73885293e-02,
   6.28131304e-11, -4.43534392e-04], [ 6.22487886e-11, -3.38511692e-14, -2.61420053e-13,  6.28131304e-11,
   2.27674378e-19, -1.54487340e-12], [ 1.48144365e-02,  5.85014193e-06, -1.19449106e-05, -4.43534392e-04,
  -1.54487340e-12,  1.77988288e-04]]
pcov = np.asarray(pcov)

In [ ]:
params = np.asarray([65,0.02,0.1,0.05,2e-9,0.96]) 
chains,chi_sq,p = MCMC(PP,get_spectrum,params,errPP,5000,pcov)

In [ ]:
file_name = 'mcmc_data_chains.txt'
file_name_1 = 'mcmc_data_chi_sq.txt'
file_name_2 = 'mcmc_data_params.txt'

np.savetxt(file_name,chains)
np.savetxt(file_name_1,chi_sq)
np.savetxt(file_name_2,p)